In [1]:
from z3 import *
from itertools import combinations
from itertools import permutations
from itertools import product

In [51]:
board=[[1,2,3,4],0,4,[2,3,4,1],0,4,[3,4,1,2],0,4]

In [69]:
board=[[1,1,2,2],0,1,[3,4,4,1],1,1,[5,3,5,1],1,1,[6,6,3,1],0,1,[2,4,5,3],1,3]

In [62]:
board=[[5,8,6,3,4],0,[7,1,6,1,8],0,[9,7,3,2,1],1]

In [9]:
board=[[9,0,3,4,2],2,[7,0,7,9,4],0,[3,9,4,5,8],2,[3,4,1,0,9],1,[5,1,5,4,5],2,[1,2,5,3,1],1]

In [64]:
board=[[5,6,1,6,1,8,5,6,5,0,5,1,8,2,9,3],2,[3,8,4,7,4,3,9,6,4,7,2,9,3,0,4,7],1,[5,8,5,5,4,6,2,9,4,0,8,1,0,5,8,7],3,[9,7,4,2,8,5,5,5,0,7,0,6,8,3,5,3],3,[4,2,9,6,8,4,9,6,4,3,6,0,7,5,4,3],3,[3,1,7,4,2,4,8,4,3,9,4,6,5,8,5,8],1,[4,5,1,3,5,5,9,0,9,4,1,4,6,1,1,7],2,[7,8,9,0,9,7,1,5,4,8,9,0,8,0,6,7],3,[8,1,5,7,3,5,6,3,4,4,1,1,8,4,8,3],1,[2,6,1,5,2,5,0,7,4,4,3,8,6,8,9,9],2,[8,6,9,0,0,9,5,8,5,1,5,2,6,2,5,4],3,[6,3,7,5,7,1,1,9,1,5,0,7,7,0,5,0],1,[6,9,1,3,8,5,9,1,7,3,1,2,1,3,6,0],1,[6,4,4,2,8,8,9,0,5,5,0,4,2,7,6,8],2,[2,3,2,1,3,8,6,1,0,4,3,0,3,8,4,5],0,[2,3,2,6,5,0,9,4,7,1,2,7,1,4,4,8],2,[5,2,5,1,5,8,3,3,7,9,6,4,4,3,2,2],2,[1,7,4,8,2,7,0,4,7,6,7,5,8,2,7,6],3,[4,8,9,5,7,2,2,6,5,2,1,9,0,3,0,6],1,[3,0,4,1,6,3,1,1,1,7,2,2,4,6,3,5],3,[1,8,4,1,2,3,6,4,5,4,3,2,4,5,8,9],3,[2,6,5,9,8,6,2,6,3,7,3,1,6,8,6,7],2]

In [3]:
# px10 matrix of booleans where p is the number of positions/length of the sequence
P = [[Bool(f'p_{p}_{n}') for n in range(10)] for p in range (1,len(board[0])+1)]
s = Solver()
# One digit must be true per position:
for pos in range(len(P)):
    s.add(Or(P[pos]))
    # No more than one digit can be true per position:
    for digit in range(10):
        for i in range(digit+1,10):
            s.add(Not(And(P[pos][digit],P[pos][i])))
for i in range(0,len(board),2):
    b = [] # List of relevant booleans for the current line of the board
    aux = []
    for j in range(len(board[i])):
        b.append(P[j][board[i][j]])
    if board[i+1] == 0:
        s.add(Not(Or(b))) # These variables must necessarily be false since no position was correct
    else:
        comb = list(combinations(b,board[i+1])) # List of tuples of length k, where k is the number of correct positions
        for tuples in comb:
            other = [ x for x in b if x not in tuples ] # Complement set of current tuple
            aux.append(And(And(tuples),Not((Or(other))))) # If current tuple is true, all other numbers are false
        s.add(Or(aux)) # One of the previous conditions must hold from the set of all tuples
print('Finding all solutions...')
count = 1
found = False
while s.check() == sat:
    m = s.model()
    solution = []
    # Search boolean matrix for trues to present solution:
    for pos in range(len(P)):
        for digit in range(10):
            if m[P[pos][digit]]:
                solution.append(digit)
    print(f'Solution {count}: {solution}')
    count += 1
    found = True
    # Deny this solution in the model to search for others:
    for pos in range(len(P)):
        s.add(Not(P[pos][solution[pos]]))
if found == False:
    print('No solutions found')

TypeError: object of type 'int' has no len()

In [34]:
s=Solver()
for i in range(0,len(board),3):
    b = [] # List of relevant booleans for the current line of the board
    aux = []
    for j in range(len(board[i])):
        b.append(P[j][board[i][j]])
    if board[i+1]==0:
        s.add(Not(Or(b)))
        if board[i+2] == 0:
            # None of the proposed numbers can be true, regardless of position:
            s.add(Not(Or([P[pos][board[i][pos]] for pos in range(len(P))])))
        else:
            combswap = list(combinations(list(enumerate(board[i])),board[i+2]))
            for tuples in combswap:
                #keep=[]
                #swap=[]
                possible=list(permutations(list(enumerate(board[i])),board[i+2]))
                for k in range(len(tuples)):
                    for move in range(len(possible)-1,-1,-1):
                        if possible[move][k] == tuples[k]:
                            possible.pop(move)
                    #keep.append(P[tuples[k][0]][tuples[k][1]])
                for groups in possible:
                    swap=[]
                    for k in range(len(groups)):
                        swap.append(P[groups[k][0]][tuples[k][1]])
                    aux.append(And(swap))
                #aux.append(And(Not(Or(keep)),And(swap)))
            s.add(Or(aux))
            aux=[]
    else:
        if board[i+2] == 0:
            comb = list(combinations(b,board[i+1])) # List of tuples of length k, where k is the number of correct positions
            for tuples in comb:
                other = [ x for x in b if x not in tuples ] # Complement set of current tuple
                aux.append(And(And(tuples),Not((Or(other))))) # If current tuple is true, all other numbers are false
            s.add(Or(aux))
#        else:
            #hard
s.check()
m=s.model()
solution = []
for pos in range(len(P)):
    for digit in range(10):
        if m[P[pos][digit]]:
            solution.append(P[pos][digit])
print(solution)
#print('Finding all solutions...')
#count = 1
#found = False
#while s.check() == sat:
#    m = s.model()
#    solution = []
    # Search boolean matrix for trues to present solution:
#    for pos in range(len(P)):
#        for digit in range(10):
#            if m[P[pos][digit]]:
#                solution.append(digit)
#    print(f'Solution {count}: {solution}')
#    count += 1
#    found = True
    # Deny this solution in the model to search for others:
#    for pos in range(len(P)):
#        print(pos,solution)
#        s.add(Not(P[pos][solution[pos]]))
#if found == False:
#    print('No solutions found')

[p_3_1]


In [31]:
combs = list(combinations(list(enumerate(board[0])),board[0+2]))
for tuples in combs:
    other = [ x for x in list(enumerate(board[0])) if x not in tuples ]
    print(other)

[(1, 1), (2, 2), (3, 2)]
[(0, 1), (2, 2), (3, 2)]
[(0, 1), (1, 1), (3, 2)]
[(0, 1), (1, 1), (2, 2)]


In [41]:
combswap = list(combinations(list(enumerate(board[0])),board[0+2]))
move = list(permutations(list(enumerate(board[0])),board[0+2]))
tuples=combswap[0]
for i in range(len(move)-1,-1,-1):
    for j in range(len(tuples)):
        if move[i][j] == tuples[j]:
            move.pop(i)
move

[((1, 1), (0, 1)),
 ((1, 1), (2, 2)),
 ((1, 1), (3, 2)),
 ((2, 2), (0, 1)),
 ((2, 2), (3, 2)),
 ((3, 2), (0, 1)),
 ((3, 2), (2, 2))]

In [8]:
x1=Bool('x1')
y1=Bool('y1')
z1=Bool('z1')
x2=Bool('x2')
y2=Bool('y2')
z2=Bool('z2')
x3=Bool('x3')
y3=Bool('y3')
z3=Bool('z3')
resolver=Solver()
cond=[]
cond.append(And(Not(Or(x1,y2)),And(x2,y3)))
cond.append(And(Not(Or(x1,y2)),And(y1,x2)))
cond.append(And(Not(Or(x1,y2)),And(y1,x3)))
resolver.add(Or(cond))
cond2=[]
cond2.append(And(Not(Or(x1,z3)),And(z1,x2)))
cond2.append(And(Not(Or(x1,z3)),And(z1,x3)))
cond2.append(And(Not(Or(x1,z3)),And(z2,x3)))
resolver.add(Or(cond2))
cond3=[]
cond3.append(And(Not(Or(y2,z3)),And(y1,z2)))
cond3.append(And(Not(Or(y2,z3)),And(y3,z1)))
cond3.append(And(Not(Or(y2,z3)),And(y3,z2)))
resolver.add(Or(cond3))
resolver.check()
#m=s.model()

sat

In [20]:
B=[[1,1,2,2],0,1]
b=[]
for j in range(len(B[0])):
    b.append(P[j][B[0][j]])
comb_black = list(combinations(b,B[0 + 1])) # List of tuples of length k, where k is the number of correct positions
comb_white = list(combinations(list(enumerate(B[0])),B[0 + 2]))
aux_black=[]
aux_white=[]
for tuples in comb_black:
    other = [ x for x in b if x not in tuples ] # Complement set of current tuple
    aux_black.append(And(And(tuples),Not((Or(other))))) # If current tuple is true, all other numbers are false
s.add(Or(aux_black))
pos = list(range(len(B[0])))
for tuples in comb_white:
    lt = len(tuples)
    ns = [tuples[i][1] for i in range (lt)]
    print(ns)
    other = [ x for x in B[0] if x not in [tuples[j][1] for j in range(lt)] ] # Complement set of current tuple
    print(other)
    other = [P[j][x] for x in other]
    print(other)
    adds = list(product(pos,ns))
    print(adds)
    print(tuples)
    aux_white.append(And(And(Or([P[x[0]][x[1]] for x in adds])),Not(Or([P[x[0]][x[1]] for x in tuples]))))# If current tuple is true, all other numbers are false


s.add(Or(aux_white))
s.check()

[1]
[2, 2]
[p_4_2, p_4_2]
[(0, 1), (1, 1), (2, 1), (3, 1)]
((0, 1),)
[1]
[2, 2]
[p_4_2, p_4_2]
[(0, 1), (1, 1), (2, 1), (3, 1)]
((1, 1),)
[2]
[1, 1]
[p_4_1, p_4_1]
[(0, 2), (1, 2), (2, 2), (3, 2)]
((2, 2),)
[2]
[1, 1]
[p_4_1, p_4_1]
[(0, 2), (1, 2), (2, 2), (3, 2)]
((3, 2),)


sat

In [57]:
s = Solver()
for pos in range(len(P)):
    s.add(Or(P[pos]))
    # No more than one digit can be true per position:
    for digit in range(10):
        for i in range(digit+1,10):
            s.add(Not(And(P[pos][digit],P[pos][i])))
for i in range(0, len(board), 3):
    b = []  # List of relevant booleans for the current line of the board
    aux = []
    ranc=False
    rans=False
    for j in range(len(board[i])):
        b.append(P[j][board[i][j]])
    #if board[i + 1] == 0:
    #    s.add(Not(Or(b)))
    #    if board[i + 2] == 0:
            # None of the proposed numbers can be true, regardless of position:
    #        s.add(Not(Or([P[pos][board[i][pos]] for pos in range(len(P))])))
    #    else:
    if board[i+2] != 0:
        combswap = list(combinations(list(enumerate(board[i])), board[i + 2]))
        for tuples in combswap:
            possible = list(permutations(list(enumerate(board[i])), board[i + 2]))
            for k in range(len(tuples)):
                for move in range(len(possible) - 1, -1, -1):
                    if possible[move][k] == tuples[k]:
                        possible.pop(move)
            for groups in possible:
                swap = []
                for k in range(len(groups)):
                    swap.append(P[groups[k][0]][tuples[k][1]])
                aux.append(And(swap))
        s.add(Or(aux))
        rans = True
    #else:
    #    if board[i + 2] == 0:
    if board[i+1] != 0:
        comb = list(
            combinations(b, board[i + 1]))  # List of tuples of length k, where k is the number of correct positions
        for tuples in comb:
            other = [x for x in b if x not in tuples]  # Complement set of current tuple
            aux.append(And(And(tuples), Not((Or(other)))))  # If current tuple is true, all other numbers are false
        s.add(Or(aux))
        ranc = True
    if not ranc:
        s.add(Not(Or(b)))
    if not rans:
        s.add(Not(Or([P[pos][board[i][pos]] for pos in range(len(P))])))




        #        else:
        #hard
#s.check()
#m = s.model()
#solution = []
#for pos in range(len(P)):
#    for digit in range(10):
#        if m[P[pos][digit]]:
#            solution.append(P[pos][digit])
#print(solution)
print('Finding all solutions...')
count = 1
found = False
while s.check() == sat:
    m = s.model()
    solution = []
 #Search boolean matrix for trues to present solution:
    for pos in range(len(P)):
        for digit in range(10):
            if m[P[pos][digit]]:
                solution.append(digit)
    print(f'Solution {count}: {solution}')
    count += 1
    found = True
 #Deny this solution in the model to search for others:
    for pos in range(len(P)):
        s.add(Not(P[pos][solution[pos]]))
if found == False:
    print('No solutions found')

Finding all solutions...
Solution 1: [5, 2, 4, 3]
Solution 2: [3, 5, 1, 4]


In [78]:
P = [[Bool(f'p_{p}_{n}') for n in range(10)] for p in range (1,len(board[0])+1)]
s=Solver()
for pos in range(len(P)):
    s.add(Or(P[pos]))
    for digit in range(10):
        for i in range(digit+1,10):
            s.add(Not(And(P[pos][digit],P[pos][i])))
for i in range(0,len(board),3):
    aux=[]
    aux2=[]
    b=[]
    for j in range(len(board[i])):
        b.append(P[j][board[i][j]])
    if board[i+1] == 0:
        s.add(Not(Or(b)))
        if board[i+2] == 0:
            s.add(Not(Or([P[pos][board[i][pos]] for pos in range(len(P))])))
        else:
            combs=list(combinations(list(enumerate(board[i])),board[i+2]))
            for tuples in combs:
                destination=list(permutations(list(enumerate(board[i])),board[i+2]))
                for elem in range(len(tuples)):
                    for move in range(len(destination)-1,-1,-1):
                        if destination[move][elem]==tuples[elem]:
                            destination.pop(move)
                for move in destination:
                    #for swap in tuples:
                    #    aux.append(Not(P[swap[0]][swap[1]]))
                    for d in range(len(move)):
                        aux.append(P[move[d][0]][tuples[d][1]])
                    aux2.append(And(aux))
            print(aux2)
            s.add(Or(aux2))
    else:
        if board[i+2] == 0:
            combc = list(combinations(b,board[i+1])) # List of tuples of length k, where k is the number of correct positions
            for tuplec in combc:
                other = [ x for x in b if x not in tuplec ] # Complement set of current tuple
                aux.append(And(And(tuplec),Not((Or(other))))) # If current tuple is true, all other numbers are false
            s.add(Or(aux))
        else:
            combc=list(combinations(list(enumerate(board[i])),board[i+1]))
            for tuplec in combc:
                aux=[]
                rest=list(enumerate(board[i]))
                for elem in tuplec:
                    rest.remove(elem)
                for j in range(len(rest)):
                    aux.append(Not(P[rest[j][0]][rest[j][1]]))
                combs=list(combinations(rest,board[i+2]))
                for tuples in combs:
                    destination=list(permutations(rest,board[i+2]))
                    for elem in range(len(tuples)):
                        for move in range(len(destination)-1,-1,-1):
                            if destination[move][elem]==tuples[elem]:
                                destination.pop(move)
                        for move in destination:
                            for c in tuplec:
                                aux.append(P[c[0]][c[1]])
                            #for swap in tuples:
                            #    aux.append(Not(P[swap[0]][swap[1]]))
                            for d in range(len(move)):
                                aux.append(P[move[d][0]][tuples[d][1]])
                            aux2.append(And(aux))
            s.add(Or(aux2))
print('Finding all solutions...')
count = 1
found = False
while s.check() == sat:
    m = s.model()
    solution = []
   # Search boolean matrix for trues to present solution:
    for pos in range(len(P)):
        for digit in range(10):
            if m[P[pos][digit]]:
                solution.append(digit)
    print(f'Solution {count}: {solution}')
    count += 1
    found = True
    # Deny this solution in the model to search for others:
    for pos in range(len(P)):
        s.add(Not(P[pos][solution[pos]]))
if found == False:
    print('No solutions found')

[And(p_2_1), And(p_2_1, p_3_1), And(p_2_1, p_3_1, p_4_1), And(p_2_1, p_3_1, p_4_1, p_1_1), And(p_2_1, p_3_1, p_4_1, p_1_1, p_3_1), And(p_2_1, p_3_1, p_4_1, p_1_1, p_3_1, p_4_1), And(p_2_1, p_3_1, p_4_1, p_1_1, p_3_1, p_4_1, p_1_2), And(p_2_1, p_3_1, p_4_1, p_1_1, p_3_1, p_4_1, p_1_2, p_2_2), And(p_2_1,
    p_3_1,
    p_4_1,
    p_1_1,
    p_3_1,
    p_4_1,
    p_1_2,
    p_2_2,
    p_4_2), And(p_2_1,
    p_3_1,
    p_4_1,
    p_1_1,
    p_3_1,
    p_4_1,
    p_1_2,
    p_2_2,
    p_4_2,
    p_1_2), And(p_2_1,
    p_3_1,
    p_4_1,
    p_1_1,
    p_3_1,
    p_4_1,
    p_1_2,
    p_2_2,
    p_4_2,
    p_1_2,
    p_2_2), And(p_2_1,
    p_3_1,
    p_4_1,
    p_1_1,
    p_3_1,
    p_4_1,
    p_1_2,
    p_2_2,
    p_4_2,
    p_1_2,
    p_2_2,
    p_3_2)]
[And(p_2_6), And(p_2_6, p_3_6), And(p_2_6, p_3_6, p_4_6), And(p_2_6, p_3_6, p_4_6, p_1_6), And(p_2_6, p_3_6, p_4_6, p_1_6, p_3_6), And(p_2_6, p_3_6, p_4_6, p_1_6, p_3_6, p_4_6), And(p_2_6, p_3_6, p_4_6, p_1_6, p_3_6, p_4_6, p_1_3), And(p_2_6

In [65]:
board=[[1,1,1],1,0,[2,2,2],1,0,[3,3,3],1,0]

In [67]:
board=[[1,2,3,4,5],2,3,[3,4,5,1,2],1,4,[5,1,2,3,4],2,3]

In [ ]:
12534